In [8]:
import pandas as pd
from collections import Counter
import nltk
import matplotlib
import matplotlib.pyplot as plt

file = pd.read_csv("Direct_Messages_-_Private_-_kathy_816837415288963072.csv")
file
file.drop(["AuthorID", "Attachments", "Reactions"], inplace=True, axis = 1)
file.dropna(subset=["Content"], inplace=True)
file.loc[:,"Author"] = file.loc[:,"Author"].str[:-5]
file

,Author,Date,Content
0,echanted,03/03/2021 5:01 PM,whats good
2,echanted,03/03/2021 5:01 PM,found a bunch of old midterms and the answers ...
3,echanted,03/03/2021 5:01 PM,have fun and you're welcome
4,kathy,03/03/2021 5:01 PM,omg lol
5,echanted,03/03/2021 5:02 PM,just keep it lowkey hahaha idk if im supposed ...
...,...,...,...
13064,echanted,12/04/2022 5:38 PM,LOL
13065,echanted,12/04/2022 5:38 PM,it's called MOSS
13066,echanted,12/04/2022 5:38 PM,by stanford
13067,kathy,12/04/2022 5:38 PM,lmao ok tyty


In [3]:
users = file.loc[:,"Author"].unique()
user1 = users[0]
user2 = users[1]
u1 = file.loc[file["Author"] == user1]
u2 = file.loc[file["Author"] == user2]

In [4]:
df = file.loc[:,["Author", "Date"]]
df["Date"] = pd.to_datetime(df["Date"])
df['block'] = ((df.Author != df.Author.shift()).cumsum())
mean_response_time = df.drop_duplicates('block', keep='first')['Date'].diff().mean()

df['response_time'] = df.drop_duplicates('block', keep='first')['Date'].diff()
gr = df.groupby('Author').response_time
mean_response_times = (gr.sum() / gr.count())/pd.Timedelta(hours=1)
mean_response_times

Author
echanted    0.990725
kathy       3.964181
Name: response_time, dtype: float64

In [5]:
stopwords = nltk.corpus.stopwords.words('english')
# RegEx for stopwords
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
words = (u1.Content
           .str.lower()
           .replace([r'\'|=|\>|\<|u|r|\"|\:|\,|\.', r'\|', RE_stopwords], ['', ' ', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rslt1 = pd.DataFrame(Counter(words).most_common(50),
                    columns=['Word', 'Frequency'])


words = (u2.Content
           .str.lower()
           .replace([r'\'|=|\>|\<|u|r|\"|\:|\,|\.|\`', r'\|', RE_stopwords], ['', ' ', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rslt2 = pd.DataFrame(Counter(words).most_common(50),
                    columns=['Word', 'Frequency'])

rslt1

,Word,Frequency
0,yo,710
1,fo,648
2,bt,540
3,’,515
4,like,512
5,im,446
6,yeah,417
7,think,405
8,jst,361
9,get,361


In [7]:
pd.concat([rslt2, pd.Series({"Word":user1, "Frequency":mean_response_times[user2]}).to_frame().T])

,Word,Frequency
0,fo,521
1,lol,472
2,bt,390
3,jst,356
4,ya,316
5,im,314
6,like,298
7,’,244
8,one,231
9,oh,202
